<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
test

In [ ]:
fs1
using System;
using System.Collections.Generic;
using System.ComponentModel;
using System.Data;
using System.Drawing;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Windows.Forms;
using System.Threading;
using Uet_Utils;
using System.IO;
using System.Runtime.InteropServices;

namespace Audio_Recognition
{

    // model https://huggingface.co/ggerganov/whisper.cpp/tree/main
    public partial class Form1 : Form
    {
        Audio_Recognition_Utils _audio_recognition = null;

        public Form1()
        {
            InitializeComponent();
            _audio_recognition = new Audio_Recognition_Utils(Txt_Model_Path.Text, none_silence_ratio: 0.3f, silence_level_threshold: 1200);
        }
        private void btn_Selected_File_Click(object sender, EventArgs e)
        {
            using (OpenFileDialog ofd = new OpenFileDialog())
            {
                if (!string.IsNullOrEmpty(Txt_Model_Path.Text))
                {
                    ofd.InitialDirectory = (new FileInfo(Txt_Model_Path.Text)).DirectoryName;
                }
                if (ofd.ShowDialog() == DialogResult.OK)
                {
                    Txt_Model_Path.Text = ofd.FileName;
                }
                _audio_recognition.Audio_Model_File_Path = Txt_Model_Path.Text;
            }
        }

        private void Btn_Model_Load_Click(object sender, EventArgs e)
        {
            try
            {
                if (_audio_recognition.Init())
                    MessageBox.Show("Model is loaded", "Information", MessageBoxButtons.OK, MessageBoxIcon.Information);
                else
                {
                    MessageBox.Show(_audio_recognition.Error_Message, "Error", MessageBoxButtons.OK, MessageBoxIcon.Error);
                }
            }
            catch (Exception _ex)
            {
                MessageBox.Show(_ex.Message, "Error", MessageBoxButtons.OK, MessageBoxIcon.Error);
            }

        }

        private async void Btn_Record_Click(object sender, EventArgs e)
        {
            if (!await _audio_recognition.Run(Process_Type.Start))
            {
                MessageBox.Show(_audio_recognition.Error_Message, "Error", MessageBoxButtons.OK, MessageBoxIcon.Error);
            }
        }

        private async void Btn_Stop_Click(object sender, EventArgs e)
        {
            _is_stream = false;
            bool _run_result = await _audio_recognition.Run(Process_Type.Stop);
            BeginInvoke(new Action(() =>
            {
                lbl_Text.Text = _audio_recognition.Recognition_Text;
                lbl_Text.Refresh();
                lbl_Text.Update();
            }));
        }
        bool _is_stream = false;
        private async void Btn_Stream_Click(object sender, EventArgs e)
        {
            _is_stream = true;
            await _audio_recognition.Run(Process_Type.Stream);
            await Task.Run(() =>
            {
                Call_Recognition_Text();
            });
            //Thread _thread = new Thread(Call_Recognition_Text);
            //_thread.IsBackground = true;
            //_thread.Start();
        }

        private void Call_Recognition_Text()
        {
            while (_is_stream)
            {
                if (_audio_recognition.Is_Completed)
                {
                    BeginInvoke(new Action(() =>
                    {
                        lbl_Text.Text = _audio_recognition.Recognition_Text;
                        lbl_Text.Refresh();
                        lbl_Text.Update();
                    }));
                    _audio_recognition.Is_Completed = false;
                }
                Thread.Sleep(100);
            }
        }
        private void Form1_FormClosing(object sender, FormClosingEventArgs e)
        {
            _audio_recognition.Dispose();
        }
        [DllImport("user32.dll")]
        public static extern bool SetCursorPos(int X, int Y);

        [DllImport("user32.dll")]
        [return: MarshalAs(UnmanagedType.Bool)]
        public static extern bool GetCursorPos(out POINT lpPoint);

        [StructLayout(LayoutKind.Sequential)]
        public struct POINT
        {
            public int X;
            public int Y;
        }
        public static void MoveMouse(int startX, int startY, int endX, int endY, int speed)
        {
            int currentX = startX;
            int currentY = startY;

            int deltaX = endX - startX;
            int deltaY = endY - startY;

            double distance = Math.Sqrt(deltaX * deltaX + deltaY * deltaY);
            double step = speed / distance;

            double t = 0.0;
            while (t <= 1.0)
            {
                t += step;
                int newX = (int)(startX + t * deltaX);
                int newY = (int)(startY + t * deltaY);
                SetCursorPos(newX, newY);
                Thread.Sleep(20); // Điều chỉnh thời gian ngủ để kiểm soát tốc độ
            }

            // Đảm bảo chuột ở đúng vị trí cuối cùng
            SetCursorPos(endX, endY);
        }
        private void Form1_Load(object sender, EventArgs e)
        {
            //// Tọa độ bắt đầu (A)
            //int startX = 10;
            //int startY = 10;

            //// Tọa độ kết thúc (B)
            //int endX = 800;
            //int endY = 600;

            //// Tốc độ di chuyển
            //int speed = 1; // Đơn vị tùy chọn, có thể điều chỉnh cho phù hợp

            //// Di chuyển chuột
            //MoveMouse(startX, startY, endX, endY, speed);


            //double x = Tools.Compare_2_Word("hello", "hel lo");
            //x = Tools.Compare_2_Word("left", "turn left");
            //x = Tools.Compare_2_Word("left", "hel right");
            int[] _out = new int[1];
            //Eye_Tracking _eye_tracking = new Eye_Tracking("resources/haarcascade_frontalface_default.xml", "resources/haarcascade_eye_tree_eyeglasses.xml");
            //_eye_tracking.Init();
            //_eye_tracking.Detection_Example(@"nn\20160910_012225.jpg", out _out);
            //_eye_tracking.Detection_Example(@"nn\122712673.jpg", out _out);
        }

        private void button1_Click(object sender, EventArgs e)
        {
            ////WaveFileWriter.CreateWaveFile("file.wav", )
            //using (WaveFileWriter writer = new WaveFileWriter("file.wav", new WaveFormat(16000, 16, 1)))
            //{
            //    writer.Write(_list_byte.ToArray(), 0, _list_byte.Count);
            //}
        }

        private async void button1_Click_1(object sender, EventArgs e)
        {
            Eye_Tracker _eye_tracking = new Eye_Tracker("resources/face_detection_yunet_2023mar.onnx");
            ////Eye_Tracking _eye_tracking = new Eye_Tracking("resources/haarcascade_frontalface_default.xml", "resources/haarcascade_eye_tree_eyeglasses.xml");
            _eye_tracking.Init();
            ////await _eye_tracking.Run(Process_Type.Stream);

            _eye_tracking.Detection(@"nn\122744864.jpg");

        }
    }
}

In [ ]:
// Eye_Tracker.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Runtime.CompilerServices;
using System.Text;
using System.Threading.Tasks;
using System.Threading;
using Emgu.CV;
using Emgu.CV.CvEnum;
using Emgu.CV.Dnn;
using Emgu.CV.Structure;
using System.Drawing;

namespace Uet_Utils
{
    // EmguCV
    // refer: https://github.com/mawax/face-detection-yunet/blob/master/Program.cs
    public class Eye_Tracker : Utils_Base
    {
        private VideoCapture _cam = null;
        private string _feature_face_file_path = "resources/face_detection_yunet_2023mar.onnx";
        private string _error_message = "";
        private int _hw_device_number = 0;
        private Mat _captured_image = new Mat();
        private bool _is_active = false;
        private string _window_name = "Face detection (Press any key to close)";
        private bool _renden_confidence = true;
        private float _confidence_score = 0.8f;
        private int _sleep_speed = 5;
        private double _movement_scale = 1920 / 200;
        /// <summary>
        /// Khởi tạo Yunet model.
        /// </summary>
        /// <param name="inputSize"></param>
        /// <returns></returns>
        private FaceDetectorYN InitializeFaceDetectionModel(Size inputSize) => new FaceDetectorYN(
            model: _feature_face_file_path,
            config: string.Empty,
            inputSize: inputSize,
            scoreThreshold: _confidence_score,
            nmsThreshold: 0.3f,
            topK: 5000,
            backendId: Emgu.CV.Dnn.Backend.Default,
            targetId: Target.Cpu);
        private FaceDetectorYN _model = null;

        #region Public Properties
        /// <summary>
        /// Index của thiết bị Audio sẽ sử dụng. (Mặc định: 0)
        /// </summary>
        public int HW_Device_Number { get => _hw_device_number; set => _hw_device_number = value; }
        /// <summary>
        /// Đường dẫn tới file Feature Face
        /// </summary>
        public string Feature_Face_File_Path { get => _feature_face_file_path; set => _feature_face_file_path = value; }

        public string Error_Message { get => _error_message; set => _error_message = value; }
        /// <summary>
        /// Giá trị độ tin cậy.
        /// </summary>
        public float Confidence_Score { get => _confidence_score; set => _confidence_score = value; }

        public int Sleep_Speed { get => _sleep_speed; set => _sleep_speed = value; }

        public double Movement_Scale { get => _movement_scale; set => _movement_scale = value; }
        #endregion

        /// <summary>
        /// Khởi tạo một đối tượng Eye Tracker.
        /// </summary>
        /// <param name="feature_face_file_path">Đường dẫn tới file Feature Face</param>
        /// <param name="hw_device_number">H/W index</param>
        public Eye_Tracker(string feature_face_file_path, int hw_device_number = 0, float _confidence_score = 0.8f)
        {
            this._feature_face_file_path = feature_face_file_path;
            this._hw_device_number = hw_device_number;
            this._confidence_score = _confidence_score;
        }

        public bool Is_Completed { get => throw new NotImplementedException(); set => throw new NotImplementedException(); }

        public string Recognition_Text => throw new NotImplementedException();

        public bool Dispose()
        {
            try
            {
                if (_cam != null)
                {
                    if (_cam.IsOpened) _cam.Stop();
                    _cam.Dispose();
                }
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }

        public bool Init()
        {
            try
            {
                if (_model != null)
                {
                    _model.Dispose();
                }
                _model = InitializeFaceDetectionModel(new Size(640, 480));
                if (_cam != null)
                {
                    if (_cam.IsOpened) _cam.Stop();
                    _cam.Dispose();
                    _cam = null;
                }
                _cam = new VideoCapture(this._hw_device_number);
                //var x  = _cam.Get(VideoCaptureProperties.Exposure);
                _cam.Set(CapProp.Contrast, 60);
                _cam.Set(CapProp.Brightness, 150);
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            };
        }

        public async Task<bool> Run(Process_Type process_type)
        {
            try
            {
                switch (process_type)
                {
                    case Process_Type.Start:
                        _is_active = false;
                        _captured_image.Dispose();
                        _captured_image = _cam.QueryFrame();
                        return _captured_image != null && !_captured_image.IsEmpty;
                    case Process_Type.Stop:
                        _is_active = false;
                        _cam.Stop();
                        break;
                    default:
                        _is_active = true;
                        var _out_offset = new int[2];
                        var _old_position = new Point(0, 0);
                        var _curr_position = new Point(0, 0);
                        await Task.Run(() =>
                        {
                            while (_is_active && _cam.IsOpened)
                            {
                                //_cam.Read(_captured_image);
                                _captured_image = _cam.QueryFrame();
                                // stuff here
                                Detection(_captured_image, _old_position, out _out_offset, out _curr_position);
                                _old_position = _curr_position;
                                Tools.Set_Mouse_Offset_Position(_out_offset[0], _out_offset[1], _movement_scale);
                                Thread.Sleep(_sleep_speed);
                            }
                        });
                        break;
                }
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }

        public bool Detection(Mat input_mat, Point old_position, out int[] out_offset, out Point curr_point, bool is_show_rect = true)
        {
            out_offset = new int[2];
            curr_point = new Point(0, 0);
            try
            {
                //input_mat.SaveImage(string.Format("{0}.jpg", DateTime.Now.ToString("hhmmssfff")));
                Land_Marks _landmarks = new Land_Marks();
                if (!Predict(input_mat, out _landmarks, is_show_rect)) return  false;

                curr_point = _landmarks.Nose;
                if (old_position.X == 0 && old_position.Y == 0)
                {
                    out_offset[0] = 0;
                    out_offset[1] = 0;
                }
                else
                {
                    out_offset[0] = curr_point.X - old_position.X;
                    out_offset[1] = curr_point.Y - old_position.Y;
                }

                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                out_offset[0] = 0;
                out_offset[1] = 0;
                return false;
            }
        }


        /// <summary>
        /// Vẽ LandMarks của khuôn mặt (Bao gồm 5 điểm).
        /// </summary>
        /// <param name="frame">Ma trận ảnh đầu vào</param>
        /// <param name="face_index">Index khuôn mặt</param>
        /// <param name="faces_data">Dữ liệu tiên đoán</param>
        void Draw_Face_LandMarks(Mat frame, int face_index, float[,] faces_data)
        {
            var _land_mark_color = new MCvScalar[]
            {
                new MCvScalar(255, 0, 0),   // right eye
                new MCvScalar(0, 0, 255),   // left eye
                new MCvScalar(0, 255, 0),   // nose tip
                new MCvScalar(255, 0, 255), // right mouth corner
                new MCvScalar(0, 255, 255)  // left mouth corner
            };

            for (var _land_mark = 0; _land_mark < 5; _land_mark++)
            {
                var _x = (int)faces_data[face_index, 4 + _land_mark * 2];
                var _y = (int)faces_data[face_index, 4 + _land_mark * 2 + 1];
                CvInvoke.Circle(frame, new Point(_x, _y), 2, _land_mark_color[_land_mark], -1);
            }
        }
        /// <summary>
        /// Vẽ bounding box hình chữ nhật quanh khuôn mặt.
        /// </summary>
        /// <param name="frame">Matrix ảnh đầu vào</param>
        /// <param name="x">Tọa độ X</param>
        /// <param name="y">Tọa độ Y</param>
        /// <param name="width">Chiều rộng</param>
        /// <param name="height">Chiều dài</param>
        void Draw_Face_Rectangle(Mat frame, int x, int y, int width, int height)
        {
            var face_rect = new Rectangle(x, y, width, height);
            CvInvoke.Rectangle(frame, face_rect, new MCvScalar(0, 255, 0), 1);
        }
        /// <summary>
        /// Vẽ score giá trị tiên đoán.
        /// </summary>
        /// <param name="frame">Matrix ảnh đầu vào</param>
        /// <param name="x">Tọa độ X</param>
        /// <param name="y">Tọa độ Y</param>
        /// <param name="confidence">Giá trị Score</param>
        void Draw_Confidence_Text(Mat frame, int x, int y, float confidence)
        {
            CvInvoke.PutText(frame, $"{confidence:N4}", new Point(x, y), FontFace.HersheyComplex, 0.3, new MCvScalar(0, 0, 255), 1);
        }
        /// <summary>
        /// Hàm vẽ bounding, score chính.
        /// </summary>
        /// <param name="frame">Ma trận ảnh đầu vào</param>
        /// <param name="faces">Ma trận giá trị tiên đoán</param>
        /// <param name="renderConfidence">Score độ tin cậy</param>
        void Draw_Detected_Faces(Mat frame, Mat faces, bool renderConfidence)
        {
            if (faces.Rows <= 0)
            {
                return;
            }

            // facesData is multidimensional array.
            // The first dimension is the index of the face, the second dimension is the data for that face.
            // The data for each face is 15 elements long:
            //  - the first 4 elements are the bounding box of the face (x, y, width, height)
            //  - the next 10 elements are the x and y coordinates of 5 facial landmarks:
            //      right eye, left eye, nose tip, right mouth corner, left mouth corner
            //  - the last element is the confidence score
            var facesData = (float[,])faces.GetData(jagged: true);

            for (var i = 0; i < facesData.GetLength(0); i++)
            {
                Draw_Face_Rectangle(frame, (int)facesData[i, 0], (int)facesData[i, 1], (int)facesData[i, 2], (int)facesData[i, 3]);
                Draw_Face_LandMarks(frame, i, facesData);

                if (renderConfidence)
                {
                    Draw_Confidence_Text(frame, (int)facesData[i, 0], (int)facesData[i, 1] - 5, facesData[i, 14]);
                }
            }
        }
        /// <summary>
        /// Template function
        /// </summary>
        /// <param name="image_path"></param>
        public void Detection(string image_path)
        {
            var input_mat = CvInvoke.Imread(image_path, ImreadModes.Color);
            //CvInvoke.Resize(input_mat, input_mat, new Size(480, 680));
            _model.InputSize = input_mat.Size;
            var _faces = new Mat();
            _model.Detect(input_mat, _faces);
            Draw_Detected_Faces(input_mat, _faces, _renden_confidence);
            CvInvoke.Imshow(_window_name, input_mat);

            return;
        }

        /// <summary>
        /// Phát hiện Landmark khuôn mặt.
        /// </summary>
        /// <param name="input_image">Matrix ảnh đầu vào</param>
        /// <param name="_return_values">Giá trị trả về</param>
        /// <param name="is_show_rect">Lựa chọn vẽ Landmark lên hình ảnh. Chế độ Debug</param>
        /// <returns></returns>
        private bool Predict(Mat input_image, out Land_Marks _return_values, bool is_show_rect = false)
        {
            _return_values = new Land_Marks();
            try
            {
                if (_model == null || input_image == null || input_image.IsEmpty) return false;
                _model.InputSize = input_image.Size;
                var _faces = new Mat();
                _model.Detect(input_image, _faces);

                if (is_show_rect ) {
                    Mat _temp_image = input_image.Clone();
                    Draw_Detected_Faces(_temp_image, _faces, _renden_confidence);
                    CvInvoke.Imshow(_window_name, _temp_image);
                }

                if (_faces.Rows <= 0) return false;

                var _face_data = (float[,])_faces.GetData(jagged: true);

                for (var i = 0; i < _face_data.GetLength(0); i++)
                {
                    for (var _land_mark = 0; _land_mark < 5; _land_mark++)
                    {
                        var _x = (int)_face_data[i, 4 + _land_mark * 2];
                        var _y = (int)_face_data[i, 4 + _land_mark * 2 + 1];
                        switch (_land_mark) {
                            case 0:
                                _return_values.Right_Eye = new Point(_x, _y);
                                break;
                            case 1:
                                _return_values.Left_Eye = new Point(_x, _y);
                                break;
                            case 2:
                                _return_values.Nose = new Point(_x, _y);
                                break;
                            case 3:
                                _return_values.Right_Mouse = new Point(_x, _y);
                                break;
                            case 4:
                                _return_values.Left_Mouse = new Point(_x, _y);
                                break;


                        }
                    }
                }
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }

        }
    }
    public struct Land_Marks
    {
        public Land_Marks()
        {
            Left_Eye = new Point(0, 0);
            Right_Eye = new Point(0, 0);
            Nose = new Point(0, 0);
            Left_Mouse = new Point(0, 0);
            Right_Mouse = new Point(0, 0);
        }
        public Land_Marks(Point left_eye, Point right_eye, Point nose, Point left_mouse, Point right_mouse)
        {
            Left_Eye = left_eye;
            Right_Eye = right_eye;
            Nose = nose;
            Left_Mouse = left_mouse;
            Right_Mouse = right_mouse;
        }
        public Point Left_Eye;
        public Point Right_Eye;
        public Point Nose;
        public Point Left_Mouse;
        public Point Right_Mouse;

        public void Null_Values()
        {
            Left_Eye = new Point(0, 0);
            Right_Eye = new Point(0, 0);
            Nose = new Point(0, 0);
            Left_Mouse = new Point(0, 0);
            Right_Mouse = new Point(0, 0);
        }
        public void Reset()
        {
            Left_Eye = new Point(0, 0);
            Right_Eye = new Point(0, 0);
            Nose = new Point(0, 0);
            Left_Mouse = new Point(0, 0);
            Right_Mouse = new Point(0, 0);
        }
    }
}


In [ ]:
// Eye_Tracking.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Runtime.CompilerServices;
using System.Text;
using System.Threading.Tasks;
using OpenCvSharp;
using System.Threading;
using OpenCvSharp.Dnn;
using OpenCvSharp.Face;

namespace Uet_Utils
{
    // refer: https://github.com/mawax/face-detection-yunet/blob/master/Program.cs
    public class Eye_Tracking : Utils_Base
    {
        private CascadeClassifier _face_cls = null;
        private CascadeClassifier _eye_cls = null;
        private VideoCapture _cam = null;
        private string _feature_face_file_path = "";
        private string _feature_eye_file_path = "";
        private string _error_message = "";
        private int _hw_device_number = 0;
        private Mat _captured_image = new Mat();
        private bool _is_active = false;
        private int _sleep_speed = 5;
        private double _movement_scale = 1920 / 200;

        #region Public Properties
        /// <summary>
        /// Index của thiết bị Audio sẽ sử dụng. (Mặc định: 0)
        /// </summary>
        public int HW_Device_Number { get => _hw_device_number; set => _hw_device_number = value; }
        /// <summary>
        /// Đường dẫn tới file Feature Face
        /// </summary>
        public string Feature_Face_File_Path { get => _feature_face_file_path; set => _feature_face_file_path = value; }
        /// <summary>
        /// Đường dẫn tới file Feature Eye
        /// </summary>
        public string Feature_Eye_File_Path { get => _feature_eye_file_path; set => _feature_eye_file_path = value; }
        public string Error_Message { get => _error_message; set => _error_message = value; }

        public int Sleep_Speed { get => _sleep_speed; set => _sleep_speed = value; }

        public double Movement_Scale { get => _movement_scale; set => _movement_scale = value; }
        #endregion

        /// <summary>
        /// Khởi tạo một đối tượng Eye Tracker.
        /// </summary>
        /// <param name="feature_face_file_path">Đường dẫn tới file Feature Face</param>
        /// <param name="feature_eye_file_path">Đường dẫn tới file Feature Eye</param>
        public Eye_Tracking(string feature_face_file_path, string feature_eye_file_path, int hw_device_number = 0)
        {
            this._feature_face_file_path = feature_face_file_path;
            this._feature_eye_file_path = feature_eye_file_path;
            this._hw_device_number = hw_device_number;
        }

        public bool Is_Completed { get => throw new NotImplementedException(); set => throw new NotImplementedException(); }

        public string Recognition_Text => throw new NotImplementedException();

        public bool Dispose()
        {
            try
            {
                if (_face_cls != null && !_face_cls.IsDisposed)
                    _face_cls.Dispose();
                if (_eye_cls != null && !_eye_cls.IsDisposed)
                    _eye_cls.Dispose();
                if (_cam != null && !_cam.IsDisposed) _cam.Dispose();
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }

        public bool Init()
        {
            try
            {
                if (_face_cls != null && !_face_cls.IsDisposed)
                    _face_cls.Dispose();
                _face_cls = new CascadeClassifier(this._feature_face_file_path);

                if (_eye_cls != null && !_eye_cls.IsDisposed)
                    _eye_cls.Dispose();
                _eye_cls = new CascadeClassifier(this._feature_eye_file_path);

                if (_cam != null && !_cam.IsDisposed)
                    _cam.Dispose();
                _cam = new VideoCapture(this._hw_device_number);
                //var x  = _cam.Get(VideoCaptureProperties.Exposure);
                _cam.Set(VideoCaptureProperties.Contrast, 60);
                _cam.Set(VideoCaptureProperties.Brightness, 150);
                //_cam.Set(VideoCaptureProperties.Exposure)
                //_cam.Set(VideoCaptureProperties.FrameWidth, 800);
                //_cam.Set(VideoCaptureProperties.FrameHeight, 720);
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            };
        }

        public async Task<bool> Run(Process_Type process_type)
        {
            try
            {
                switch (process_type)
                {
                    case Process_Type.Start:
                        _is_active = false;
                        return _cam.Read(_captured_image);
                    case Process_Type.Stop:
                        _is_active = false;
                        _cam.Release();
                        break;
                    default:
                        _is_active = true;
                        var _out_offset = new int[2];
                        var _old_position = new Point(0, 0);
                        var _curr_position = new Point(0, 0);
                        await Task.Run(() =>
                        {
                            while (_is_active && _cam.IsOpened())
                            {
                                _cam.Read(_captured_image);
                                Detection(_captured_image, _old_position, out _out_offset, out _curr_position);
                                _old_position = _curr_position;
                                Tools.Set_Mouse_Offset_Position(_out_offset[0], _out_offset[1], _movement_scale);
                                Thread.Sleep(_sleep_speed);
                            }
                        });
                        break;
                }
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }

        public bool Detection(Mat input_mat, Point old_position, out int[] out_offset, out Point curr_point, bool is_show_rect = true)
        {
            out_offset = new int[2];
            curr_point = new Point(0, 0);
            try
            {

                input_mat.SaveImage(string.Format("{0}.jpg", DateTime.Now.ToString("hhmmssfff")));
                var _gray_image = new Mat(input_mat.Size(), MatType.CV_8UC1);
                Cv2.CvtColor(input_mat, _gray_image, ColorConversionCodes.BGR2GRAY);
                //Cv2.Normalize(_gray_image, _gray_image, 0, 255, NormTypes.MinMax);
                //Cv2.Normalize(_gray_image, _gray_image, 0, 1000, NormTypes.MinMax);
                //_gray_image = _gray_image.Normalize(0, 255, NormTypes.MinMax);
                var _face_pos = this._face_cls.DetectMultiScale(_gray_image, 1.1, 9);
                List<Rect> _rects = new List<Rect>();
                foreach (var _rect in _face_pos)
                {
                    if (is_show_rect) Cv2.Rectangle(input_mat, _rect, new Scalar(255, 0, 0), 3);
                    var _region = new Mat(_gray_image, _rect);
                    var _eye_pos = this._eye_cls.DetectMultiScale(_region);
                    _eye_pos = _eye_pos.Select(x => x.Add(new Point(_rect.X, _rect.Y))).ToArray();
                    if (is_show_rect) foreach (var _eye in _eye_pos)
                        {
                            Cv2.Rectangle(input_mat, _eye, new Scalar(0, 255, 0), 3);
                        }
                    _rects = _rects.Concat(_eye_pos).ToList();
                }
                if (is_show_rect)
                {
                    Cv2.ImShow("img", input_mat);
                    Cv2.ImShow("img2", _gray_image);
                    Cv2.WaitKey(0);
                }

                curr_point.X = (int)_rects.Select(x => x.X).Average();
                curr_point.Y = (int)_rects.Select(x => x.Y).Average();
                if (old_position.X == 0 && old_position.Y == 0)
                {
                    out_offset[0] = 0;
                    out_offset[1] = 0;
                }
                else
                {
                    out_offset[0] = curr_point.X - old_position.X;
                    out_offset[1] = curr_point.Y - old_position.Y;
                }

                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                out_offset[0] = 0;
                out_offset[1] = 0;
                return false;
            }
        }

        public bool Detection_Example(string input_file, out int[] out_positions)
        {
            out_positions = new int[1];
            try
            {
                var input_mat = Cv2.ImRead(input_file, ImreadModes.Color);
                var _gray_image = new Mat(input_mat.Size(), MatType.CV_8UC1);
                Cv2.CvtColor(input_mat, _gray_image, ColorConversionCodes.BGR2GRAY);
                var _face_pos = this._face_cls.DetectMultiScale(_gray_image, 1.1, 4);
                foreach (var _rect in _face_pos)
                {
                    Cv2.Rectangle(input_mat, _rect, new Scalar(255, 0, 0), 3);
                    var _region = new Mat(_gray_image, _rect);
                    var _eye_pos = this._eye_cls.DetectMultiScale(_region);
                    foreach (var _sub_rect in _eye_pos)
                    {
                        Cv2.Rectangle(input_mat, _sub_rect.Add(new Point(_rect.X, _rect.Y)), new Scalar(0, 255, 0), 3);
                    }
                }
                Cv2.ImShow("img", input_mat.Resize(new Size(600, 800)));
                Cv2.WaitKey(0);
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }

        public void Dection(string image_path)
        {
            var input_mat = Cv2.ImRead(image_path, ImreadModes.Color).Resize(new Size(480, 640));
            var _dnn = CvDnn.ReadNet("resources/face_detection_yunet_2023mar.onnx");
            //var _dnn = CvDnn.ReadNetFromCaffe("resources/deploy.prototxt", "resources/res10_300x300_ssd_iter_140000_fp16.caffemodel");
            var inputBlob = CvDnn.BlobFromImage(input_mat, (double)1 / 255, new Size(320, 320), swapRB:false, crop:false);
            _dnn.SetInput(inputBlob);
            var _layers = _dnn.GetLayerNames();
            var _out_value = _dnn.Forward();
            var x = _out_value.Size();
            ProcessDetections(input_mat, _out_value);

            return;
        }
        void ProcessDetections(Mat frame, Mat detection)
        {
            float confidenceThreshold = 0.7f;
            var xxx = detection.Size();
            using var detectionMat = new Mat(detection.Size(2), detection.Size(3), MatType.CV_32F, detection.Ptr(0));
            for (int i = 0; i < detectionMat.Rows; i++)
            {
                float confidence = detectionMat.At<float>(i, 2);

                if (confidence > confidenceThreshold)
                {
                    int x = (int)(detection.At<float>(i, 3) * frame.Cols);
                    int y = (int)(detection.At<float>(i, 4) * frame.Rows);
                    int w = (int)(detection.At<float>(i, 5) * frame.Cols);
                    int h = (int)(detection.At<float>(i, 6) * frame.Rows);

                    // Vẽ hình chữ nhật xung quanh các khuôn mặt được phát hiện
                    var rect = new Rect(x, y, w, h);
                    Cv2.Rectangle(frame, rect, Scalar.Green, 5);
                }
            }
            Cv2.ImShow("img", frame);
            Cv2.WaitKey(0);
        }

        //public bool Detection_Example(Mat input_mat, out int[] out_positions)
        //{
        //    out_positions = new int[1];
        //    try
        //    {
        //        var _gray_image = new Mat(input_mat.Size(), MatType.CV_8UC1);
        //        Cv2.CvtColor(input_mat, _gray_image, ColorConversionCodes.BGR2GRAY);
        //        var _face_pos = this._face_cls.DetectMultiScale(_gray_image, 1.1, 4);
        //        foreach (var _rect in _face_pos)
        //        {
        //            Cv2.Rectangle(input_mat, _rect, new Scalar(255, 0, 0), 3);
        //            var _region = new Mat(_gray_image, _rect);
        //            var _eye_pos = this._eye_cls.DetectMultiScale(_region);
        //            foreach (var _sub_rect in _eye_pos)
        //            {
        //                Cv2.Rectangle(input_mat, _rect.Add(new Point(_rect.X, _rect.Y)), new Scalar(0, 255, 0), 3);
        //            }
        //        }
        //        Cv2.ImShow("img", input_mat);
        //        Cv2.WaitKey(0);
        //        return true;
        //    }
        //    catch (Exception)
        //    {

        //        return false;
        //    }
        //}
    }
}


In [ ]:
// Tools.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Runtime.InteropServices;
using System.Threading;

namespace Uet_Utils
{
    public enum Comparing_Method { Levenshtein, }
    public enum Mouse_Function { Left, Right, Up, Down, Left_Click, Right_Click, Double_Click }

    public static class Tools
    {
        [DllImport("user32")]
        public static extern int SetCursorPos(int x, int y);
        [DllImport("user32.dll")]
        public static extern void mouse_event(uint dwFlags, uint dx, uint dy, uint dwData, int dwExtraInfo);
        // Cấu trúc POINT để lưu trữ tọa độ chuột
        public struct POINT
        {
            public int X;
            public int Y;
        }
        [DllImport("user32.dll")]
        private static extern bool GetCursorPos(out POINT lpPoint);
        /// <summary>
        /// Thay đổi tọa độ vị trí chuột (X, Y).
        /// </summary>
        /// <param name="x">Tọa độ X</param>
        /// <param name="y">Tọa độ Y</param>
        public static void Set_Mouse_Position(int x, int y)
        {
            SetCursorPos(x, y);
        }
        /// <summary>
        /// Thay đổi tọa độ vị trí chuột, tới vị trí mới.
        /// </summary>
        /// <param name="x">Offset tọa độ X</param>
        /// <param name="y">Offset tọa độ Y</param>
        public static void Set_Mouse_Offset_Position(int x, int y, double scale = 1.0)
        {
            try
            {
                POINT _curr_pos = new POINT();
                _curr_pos.X = 0; _curr_pos.Y = 0;
                GetCursorPos(out _curr_pos);
                SetCursorPos(_curr_pos.X + (int)(x * scale), _curr_pos.Y + (int)(y * scale));
            }
            catch
            {
            }
        }
        /// <summary>
        /// Click chuột trái tại tọa độ X, Y.
        /// </summary>
        /// <param name="x">Tọa độ X</param>
        /// <param name="y">Tọa độ Y</param>
        public static void Left_Mouse_Click(int x, int y)
        {
            SetCursorPos(x, y);
            mouse_event(MOUSEEVENTF_LEFTDOWN, (uint)x, (uint)y, 0, 0);
            mouse_event(MOUSEEVENTF_LEFTUP, (uint)x, (uint)y, 0, 0);
        }
        /// <summary>
        /// Click double chuột phải tại tọa độ X, Y.
        /// </summary>
        /// <param name="x">Tọa độ X</param>
        /// <param name="y">Tọa độ Y</param>
        public static void Left_Mouse_Double_Click(int x, int y)
        {
            SetCursorPos(x, y);
            mouse_event(MOUSEEVENTF_LEFTDOWN, (uint)x, (uint)y, 0, 0);
            mouse_event(MOUSEEVENTF_LEFTUP, (uint)x, (uint)y, 0, 0);
            Thread.Sleep(100);
            mouse_event(MOUSEEVENTF_RIGHTDOWN, (uint)x, (uint)y, 0, 0);
            mouse_event(MOUSEEVENTF_RIGHTUP, (uint)x, (uint)y, 0, 0);
        }
        private const uint MOUSEEVENTF_LEFTDOWN = 0x02;
        private const uint MOUSEEVENTF_LEFTUP = 0x04;
        private const uint MOUSEEVENTF_RIGHTDOWN = 0x08;
        private const uint MOUSEEVENTF_RIGHTUP = 0x10;
        /// <summary>
        /// So sánh sự tương đồng giữa 2 chuỗi.
        /// </summary>
        /// <param name="s_string">Chuỗi 1</param>
        /// <param name="d_string">Chuỗi 2</param>
        /// <param name="comparing_method">Phương pháp so sánh</param>
        /// <returns></returns>
        public static double Compare_2_Word(string s_string, string d_string, Comparing_Method comparing_method = Comparing_Method.Levenshtein)
        {
            switch (comparing_method)
            {
                case Comparing_Method.Levenshtein:
                    return 1 - (double)Algorithms.Calculate(s_string, d_string) / (double)Math.Max(s_string.Length, d_string.Length);
                default:
                    return 99999;
            }

        }
    }
}


In [ ]:
// base.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace Uet_Utils
{
    internal interface Utils_Base
    {
        /// <summary>
        /// Biến kiểm tra kết quả. Hoàn thành tiến trình.
        /// </summary>
        public bool Is_Completed { get; set; }

        /// <summary>
        /// Biến thông báo lỗi.
        /// </summary>
        public string Error_Message { get; set; }

        /// <summary>
        /// Kết quả nhận dạng âm thanh.
        /// </summary>
        public string Recognition_Text { get; }
        /// <summary>
        /// Index HW sử dụng.
        /// </summary>
        public int HW_Device_Number { get; set; }

        /// <summary>
        /// Thời gian nghỉ giữa các tiến trình (Tương ứng tốc độ chuột).
        /// </summary>
        public int Sleep_Speed { get; set; }

        /// <summary>
        /// Thời gian nghỉ giữa các tiến trình (Tương ứng tốc độ chuột).
        /// </summary>
        public double Movement_Scale { get; set; }

        /// <summary>
        /// Khởi tạo đối tượng lớp tính năng.
        /// </summary>
        /// <returns></returns>
        public bool Init();
        /// <summary>
        /// Khởi chạy tính năng.
        /// </summary>
        /// <param name="process_type">Kiểu tính năng (Start, Stop, Stream)</param>
        /// <returns></returns>
        public Task<bool> Run(Process_Type process_type);

        // <summary>
        /// Xóa bỏ toàn bộ các đối tượng.
        /// </summary>
        /// <returns></returns>
        public bool Dispose();

    }
}


In [ ]:
// audio.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Threading;
using NAudio.Wave;
using Whisper.net;
using System.Diagnostics;

namespace Uet_Utils
{
    public enum Process_Type { Start, Stop, Stream }
    public class Audio_Recognition_Utils : Utils_Base
    {
        private readonly SemaphoreSlim _semaphore_llim = new SemaphoreSlim(1, 1);
        private string _audio_model_file_path = "";
        private int _hw_device_number = 0;
        private string _error_message = "";
        private WaveInEvent _waveSource = new WaveInEvent();
        private WhisperProcessor _processor = null;
        private List<float> _list_float = new List<float>();
        private List<byte> _list_byte = new List<byte>();
        private DateTime _last_sound_detection = DateTime.Now;
        private TimeSpan _silence_duration_threshold = TimeSpan.FromSeconds(3);
        private float _silence_level_threshold = 30;
        private float _none_silence_ratio = 0.9f;
        private bool _is_active = false;
        private bool _is_completed = false;
        private string _recognition_text = "";
        private int _sleep_speed = 5;
        private double _movement_scale = 1920 / 200;
        #region public properties
        /// <summary>
        /// Thông báo lỗi bất kỳ khi nào có lỗi phát sinh.
        /// </summary>
        public string Error_Message { get => _error_message; set => _error_message = value; }
        /// <summary>
        /// Đường dẫn tới file model Whisper.
        /// </summary>
        public string Audio_Model_File_Path { get => _audio_model_file_path; set => _audio_model_file_path = value; }
        /// <summary>
        /// Index của thiết bị Audio sẽ sử dụng. (Mặc định: 0)
        /// </summary>
        public int HW_Device_Number { get => _hw_device_number; set => _hw_device_number = value; }
        /// <summary>
        /// Ngưỡng thời gian dừng tối đa để phát hiện khoảng lặng. (Mặc định 3 giây)
        /// </summary>
        public TimeSpan Silence_Duration_Threshold { get => _silence_duration_threshold; set => _silence_duration_threshold = value; }
        /// <summary>
        /// Giá trị độ lớn âm lượng tối thiểu cần đạt. Nhỏ hơn mặc định là khoảng lặng. (Mặc định: 30)
        /// </summary>
        public float Silence_Level_Threshold { get => _silence_level_threshold; set => _silence_level_threshold = value; }
        /// <summary>
        /// Tỉ lệ số lượng giá trị lớn hơn ngưỡng khoảng lặng tối thiểu. Nhỏ hơn mặc định là khoảng lặng. (Mặc định: 0.9)
        /// </summary>
        public float None_Silence_Ratio { get => _none_silence_ratio; set => _none_silence_ratio = value; }
        public bool Is_Completed { get => _is_completed; set => _is_completed = value; }
        public string Recognition_Text { get => _recognition_text; }
        public int Sleep_Speed { get => _sleep_speed; set => _sleep_speed = value; }

        public double Movement_Scale { get => _movement_scale; set => _movement_scale = value; }
        #endregion
        /// <summary>
        /// Khởi tạo một đối tượng nhận dạng giọng nói.
        ///
        /// </summary>
        /// <param name="audio_model_file_path">Đường dẫn tới file model Whisper.</param>
        /// <param name="audio_hw_device_number">Index của thiết bị Audio sẽ sử dụng. (Mặc định: 0)</param>
        public Audio_Recognition_Utils(string audio_model_file_path, int hw_device_number = 0, float none_silence_ratio = 0.9f, float silence_level_threshold = 30)
        {
            this._audio_model_file_path = audio_model_file_path;
            this._hw_device_number = hw_device_number;
            this._none_silence_ratio = none_silence_ratio;
            this._silence_level_threshold = silence_level_threshold;
        }

        /// <summary>
        /// Nhận dạng giọng nói. Chuyển âm thanh sang hình ảnh sử dụng Whisper (Open AI).
        /// </summary>
        /// <param name="_list_float">Buffer Audio</param>
        /// <param name="processor">Core processer</param>
        /// <returns></returns>
        private async Task<bool> Recognition(List<float> _list_float, WhisperProcessor processor)
        {
            _recognition_text = "";
            try
            {
                _error_message = "";
                await foreach (var result in processor.ProcessAsync(_list_float.ToArray()))
                {
                    //Console.WriteLine($"{result.Start}->{result.End}: {result.Text}");
                    _recognition_text += result.Text + ", ";
                }
                return true;
            }
            catch (Exception _ex)
            {
                _error_message += _ex.Message;
                return false;
            }
            finally
            {
                _is_completed = true;
            }
        }
        /// <summary>
        /// Sự kiện thu nhận âm thanh.
        /// </summary>
        /// <param name="sender"></param>
        /// <param name="e"></param>
        async void waveSource_DataAvailable(object sender, WaveInEventArgs e)
        {
            await _semaphore_llim.WaitAsync();
            try
            {
                var _value = Math.Abs(BitConverter.ToInt16(e.Buffer, (e.BytesRecorded - 2)));
                int _silence_count = 0;
                for (int index = 0; index < e.BytesRecorded; index += 2)
                {
                    short sample = BitConverter.ToInt16(e.Buffer, index);
                    if (Math.Abs(sample) < this._silence_level_threshold)
                    {
                        _silence_count++;
                    }
                }

                // If more than 90% of the samples are below the threshold, consider it as silence
                if ((e.BytesRecorded / 2) * this._none_silence_ratio > _silence_count)
                {
                    _is_completed = false;
                    _last_sound_detection = DateTime.Now;

                    byte[] buffer = e.Buffer;
                    for (int index = 0; index < e.BytesRecorded; index += 2)
                    {
                        short sample = (short)((buffer[index + 1] << 8) |
                                                buffer[index]);
                        float sample32 = sample / 32768f;
                        _list_float.Add(sample32);
                    }
                    _list_byte = _list_byte.Concat(buffer.ToList()).ToList();
                }
                else
                {
                    if (DateTime.Now - _last_sound_detection > _silence_duration_threshold && _is_active && _list_byte.Count() > 0)
                    {
                        Debug.Print("Silence detected.");
                        // stuff here
                        //var _list_temp = _list_float.ConvertAll(x => x).ToList();
                        await Recognition(_list_float, _processor);
                        _list_byte.Clear();
                        _list_float.Clear();
                        // Take action on silence detection, e.g., stop recording, log, etc.
                    }
                }
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
            }
            finally
            {
                _semaphore_llim.Release();
            }
        }
        /// <summary>
        /// Khởi tạo model, kết nối Audio HW.
        /// </summary>
        /// <returns></returns>
        public bool Init()
        {
            try
            {
                if (_waveSource != null)
                {
                    _waveSource.Dispose();
                    _waveSource = null;
                }
                if (_processor != null)
                {
                    _processor.Dispose();
                    _processor = null;
                }

                _waveSource.DeviceNumber = this._hw_device_number;
                _waveSource.WaveFormat = new WaveFormat(16000, 1);
                var whisperFactory = WhisperFactory.FromPath(this._audio_model_file_path);
                _processor = whisperFactory.CreateBuilder()
                    .WithLanguage("auto")
                    .Build();
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }
        /// <summary>
        /// Khởi chạy thu âm và nhận dạng giọng nói.
        /// </summary>
        /// <param name="process_type">Phương thức vận hành</param>
        /// <returns></returns>
        public async Task<bool> Run(Process_Type process_type)
        {
            try
            {
                switch (process_type)
                {
                    case Process_Type.Start:
                        _is_active = false;
                        _list_float.Clear();
                        _list_byte.Clear();
                        _waveSource.StartRecording();
                        _waveSource.DataAvailable += waveSource_DataAvailable;
                        break;
                    case Process_Type.Stop:
                        _is_active = false;
                        _waveSource.StopRecording();
                        _waveSource.DataAvailable -= waveSource_DataAvailable;
                        await Recognition(_list_float, _processor);
                        _list_float.Clear();
                        _list_byte.Clear();
                        break;
                    default:
                        _is_active = true;
                        _list_float.Clear();
                        _list_byte.Clear();
                        _waveSource.StartRecording();
                        _waveSource.DataAvailable += waveSource_DataAvailable;
                        break;
                }
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }

        public bool Dispose()
        {
            try
            {
                if (_waveSource != null)
                    _waveSource.Dispose();
                if (_processor != null)
                    _processor.Dispose();
                return true;
            }
            catch (Exception _ex)
            {
                _error_message = _ex.Message;
                return false;
            }
        }
    }
}


In [ ]:
// algo.c
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace Uet_Utils
{
    // origin source here
    // https://gist.github.com/Davidblkx/e12ab0bb2aff7fd8072632b396538560
    // Thank you Davidblkx
    internal class Algorithms
    {
        /// <summary>
        ///     Calculate the difference between 2 strings using the Levenshtein distance algorithm
        /// </summary>
        /// <param name="source1">First string</param>
        /// <param name="source2">Second string</param>
        /// <returns></returns>
        public static int Calculate(string source1, string source2) //O(n*m)
        {
            var source1Length = source1.Length;
            var source2Length = source2.Length;

            var matrix = new int[source1Length + 1, source2Length + 1];

            // First calculation, if one entry is empty return full length
            if (source1Length == 0)
                return source2Length;

            if (source2Length == 0)
                return source1Length;

            // Initialization of matrix with row size source1Length and columns size source2Length
            for (var i = 0; i <= source1Length; matrix[i, 0] = i++) { }
            for (var j = 0; j <= source2Length; matrix[0, j] = j++) { }

            // Calculate rows and collumns distances
            for (var i = 1; i <= source1Length; i++)
            {
                for (var j = 1; j <= source2Length; j++)
                {
                    var cost = (source2[j - 1] == source1[i - 1]) ? 0 : 1;

                    matrix[i, j] = Math.Min(
                        Math.Min(matrix[i - 1, j] + 1, matrix[i, j - 1] + 1),
                        matrix[i - 1, j - 1] + cost);
                }
            }
            // return result
            return matrix[source1Length, source2Length];
        }
    }
}
